# Machine Learning Lab Project - Credit Card Overdue Likelihood Prediction
##  Business Understanding

## Data Understanding
The data for this task is taken from [this](https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction) kaggle dataset. The kaggle page provides two `.csv` files:
- application_record.csv
- credit_record.csv

On a simple level, `application_record.csv` contains the customer data and `credit_record.csv` contains the customers credit history. The specific content is now investigated further.

For beeing able to analyse the datasets, the necessary libraries are imported first:

In [1]:
import pandas as pd

In this next step the two `.csv` files are loaded into a pandas datafram. This enables an analysis with the full pandas funcionality, which makes the data understanding process way easier.

In [2]:
customer_df = pd.read_csv("Data/application_record.csv")
credit_df = pd.read_csv("Data/credit_record.csv")

### application_record.csv
First, it is important to analyse the columns of the dataset:

In [3]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   438557 non-null  int64  
 1   CODE_GENDER          438557 non-null  object 
 2   FLAG_OWN_CAR         438557 non-null  object 
 3   FLAG_OWN_REALTY      438557 non-null  object 
 4   CNT_CHILDREN         438557 non-null  int64  
 5   AMT_INCOME_TOTAL     438557 non-null  float64
 6   NAME_INCOME_TYPE     438557 non-null  object 
 7   NAME_EDUCATION_TYPE  438557 non-null  object 
 8   NAME_FAMILY_STATUS   438557 non-null  object 
 9   NAME_HOUSING_TYPE    438557 non-null  object 
 10  DAYS_BIRTH           438557 non-null  int64  
 11  DAYS_EMPLOYED        438557 non-null  int64  
 12  FLAG_MOBIL           438557 non-null  int64  
 13  FLAG_WORK_PHONE      438557 non-null  int64  
 14  FLAG_PHONE           438557 non-null  int64  
 15  FLAG_EMAIL       

As can be seen above, the dataset consists of 17 columns, containing numeral as well as textual data. It also seems as if there is already an unique identifier for every customer in the column `ID`. A concrete description fo these different columns can be retrieved from the datasets [kaggle page](https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction/data):

|Feature name|Explanation|Remarks|
|:-----------|:----------|:------|
|ID 	     |Client number||
|CODE_GENDER |	Gender 	||
|FLAG_OWN_CAR| 	Is there a car 	||
|FLAG_OWN_REALTY| 	Is there a property|| 	
|CNT_CHILDREN| 	Number of children 	||
|AMT_INCOME_TOTAL| 	Annual income 	||
|NAME_INCOME_TYPE| 	Income category 	||
|NAME_EDUCATION_TYPE| 	Education level ||	
|NAME_FAMILY_STATUS| 	Marital status 	||
|NAME_HOUSING_TYPE| 	Way of living 	||
|DAYS_BIRTH| 	Birthday |	Count backwards from current day (0), -1 means yesterday|
|DAYS_EMPLOYED| 	Start date of employment |	Count backwards from current day(0). If positive, it means the person currently  unemployed.|
|FLAG_MOBIL| 	Is there a mobile phone 	||
|FLAG_WORK_PHONE| 	Is there a work phone 	||
|FLAG_PHONE| 	Is there a phone 	||
|FLAG_EMAIL| 	Is there an email 	||
|OCCUPATION_TYPE| 	Occupation 	||
|CNT_FAM_MEMBERS| 	Family size||

Now that the purpose of the columns is clear, the actual data can be analyzed:

In [4]:
customer_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


From these first five entries several things can be observed:
1. The ID does not start from 0 but seems to be unique.
2. For the `CODE_GENDER` the flags `F` (Female) and `M` (Male) are used.
3. For `FLAG_OWN_CAR` and `FLAG_OWN_REALTY` the flags `Y` (Yes) and `N` (No) are used.
4. For `NAME_INCOME_TYPE`, `NAME_EDUCATION_TYPE`, `NAME_FAMILY_STATUS` and `NAME_HOUSING_TYPE` are textual fields, but seem to have only a few different values.
5. `OCCUPATION_TYPE` is a textual field and the values seem to be very different ("Freetext Field").
6. For `FLAG_MOBIL`, `FLAG_WORK_PHONE`, `FLAG_PHONE` and `FLAG_EMAIL` the flags 1 (Yes) and 0 (No) are used.

Before basing the data perparation on these findings, the assumptions have to be validated:

#### 1. Unique `ID`

In [5]:
customer_df['ID'].value_counts()

ID
7137299    2
7702238    2
7282535    2
7243768    2
7050948    2
          ..
5690727    1
6621262    1
6621261    1
6621260    1
6842885    1
Name: count, Length: 438510, dtype: int64

The ouptut of the value count clearly shows that not every ID is unique. As some IDs are contained twice, the real amout of customers contained in the dataset is only 438510.
#### 2. Gender

In [6]:
customer_df['CODE_GENDER'].value_counts()

CODE_GENDER
F    294440
M    144117
Name: count, dtype: int64

For the gender the assumption that only the flags `F` and `M` are used was correct.
#### 3. Flag Car / Real-Estate

In [7]:
print(customer_df['FLAG_OWN_CAR'].value_counts())
print(customer_df['FLAG_OWN_REALTY'].value_counts())

FLAG_OWN_CAR
N    275459
Y    163098
Name: count, dtype: int64
FLAG_OWN_REALTY
Y    304074
N    134483
Name: count, dtype: int64


Also for the flags for the possesion of car and real-estate the assumption that there are only the flags `Y` and `N` was correct.
#### 4. Text fields income, education, family and housing

In [8]:
customer_df['NAME_INCOME_TYPE'].value_counts()

NAME_INCOME_TYPE
Working                 226104
Commercial associate    100757
Pensioner                75493
State servant            36186
Student                     17
Name: count, dtype: int64

As can be seen above, there are five different types of income. This means the column can be encoded without any problems and can be used for the modeling later.

In [9]:
customer_df['NAME_EDUCATION_TYPE'].value_counts()

NAME_EDUCATION_TYPE
Secondary / secondary special    301821
Higher education                 117522
Incomplete higher                 14851
Lower secondary                    4051
Academic degree                     312
Name: count, dtype: int64

The education field also consists of only five types and can therefore also be used for modelling without any problems.

In [10]:
customer_df['NAME_FAMILY_STATUS'].value_counts()

NAME_FAMILY_STATUS
Married                 299828
Single / not married     55271
Civil marriage           36532
Separated                27251
Widow                    19675
Name: count, dtype: int64

Also the family status has five different values.

In [11]:
customer_df['NAME_HOUSING_TYPE'].value_counts()

NAME_HOUSING_TYPE
House / apartment      393831
With parents            19077
Municipal apartment     14214
Rented apartment         5974
Office apartment         3922
Co-op apartment          1539
Name: count, dtype: int64

The housing has six different values, which is still acceptable.
#### 5. Text field occupation type

In [12]:
print(customer_df['OCCUPATION_TYPE'].value_counts())
print("Amount of null values: ", customer_df['OCCUPATION_TYPE'].isnull().sum())

OCCUPATION_TYPE
Laborers                 78240
Core staff               43007
Sales staff              41098
Managers                 35487
Drivers                  26090
High skill tech staff    17289
Accountants              15985
Medicine staff           13520
Cooking staff             8076
Security staff            7993
Cleaning staff            5845
Private service staff     3456
Low-skill Laborers        2140
Secretaries               2044
Waiters/barmen staff      1665
Realty agents             1041
HR staff                   774
IT staff                   604
Name: count, dtype: int64
Amount of null values:  134203


Even though there are way less different values than expected (only 18), the column contains many null values, which may make it difficult to work with it.
#### 6. Contact method flags

In [13]:
print(customer_df['FLAG_MOBIL'].value_counts())
print(customer_df['FLAG_WORK_PHONE'].value_counts())
print(customer_df['FLAG_PHONE'].value_counts())
print(customer_df['FLAG_EMAIL'].value_counts())

FLAG_MOBIL
1    438557
Name: count, dtype: int64
FLAG_WORK_PHONE
0    348156
1     90401
Name: count, dtype: int64
FLAG_PHONE
0    312353
1    126204
Name: count, dtype: int64
FLAG_EMAIL
0    391102
1     47455
Name: count, dtype: int64


As expected, these columns only use the flags `1` and `0`. On top of that, column `FLAG_MOBIL` only contains the value `1`, which means all customers at least are registered with a mobile phone. Therefore, this column can be left out completely.
### credit_record.csv
Again, first analyze the columns:

In [14]:
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   ID              1048575 non-null  int64 
 1   MONTHS_BALANCE  1048575 non-null  int64 
 2   STATUS          1048575 non-null  object
dtypes: int64(2), object(1)
memory usage: 24.0+ MB


As can be seen above, this table only contains three columns. According to the [kaggle page](https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction/data), these columns mean the following:

|Feature name| 	Explanation |	Remarks|
|:-|:-|:-|
|ID |	Client number 	||
|MONTHS_BALANCE |	Record month |	The month of the extracted data is the starting point, backwards, 0 is the current month, -1 is the previous month, and so on|
|STATUS |	Status| 	0: 1-29 days past due 1: 30-59 days past due 2: 60-89 days overdue 3: 90-119 days overdue 4: 120-149 days overdue 5: Overdue or bad debts, write-offs for more than 150 days C: paid off that month X: No loan for the month|

Checking for unique `ID` values now reveals for how many customers there exists credit data:

In [15]:
len(credit_df['ID'].unique())

45985

There is only credit data for 45985 customers, that means only parts of the `customer_df` can be used.
As a last step check the values of the `STATUS` column:

In [16]:
credit_df['STATUS'].value_counts()

STATUS
C    442031
0    383120
X    209230
1     11090
5      1693
2       868
3       320
4       223
Name: count, dtype: int64

The information given on the kaggle page is correct, only the stated flags are used.
### Summary
Overall, the dataset consists of two parts: the customer data and the credit data. The customer data mostly contains information about income, job, family situation and contact methods as these are important aspects for evaluating the creditworthiness. The credit data is basically a credit history overview, showing for a given custumer and month if the credit was paid back on time. This credit data can now be used for calculating an "overdue_likelyhood" for every customer which states how likely it is for this specific customer to not pay it's credit back in time. This is an important information for a credit institute. Based on all the findings in this section the two datasets can now be prepared, connected and finally used for training a machine learning model.

## Data Preparation

Get the overdue likelyhood for the customers:

In [17]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

grouped_df = credit_df.groupby('ID')['STATUS'].value_counts().unstack(fill_value=0)

grouped_df['overdue_likelihood'] = 1 - (grouped_df['C'] / (grouped_df.sum(axis=1) - grouped_df['X']))

result_df = grouped_df.reset_index()[['ID', 'overdue_likelihood']]

print(result_df.head())

nan_count = result_df['overdue_likelihood'].isna().sum()

print(f'The number of entries with NaN values in overdue_likelihood: {nan_count}')


STATUS       ID  overdue_likelihood
0       5001711            1.000000
1       5001712            0.526316
2       5001713                 NaN
3       5001714                 NaN
4       5001715                 NaN
The number of entries with NaN values in overdue_likelihood: 4536


That means ca. 33110 customers are really usable.

Remove NaN entrys

In [18]:
result_df.dropna(subset=['overdue_likelihood'], inplace=True)
nan_count = result_df['overdue_likelihood'].isna().sum()

print(f'The number of entries with NaN values in overdue_likelihood: {nan_count}')

The number of entries with NaN values in overdue_likelihood: 0


Merge the data:

In [19]:

merged_df = pd.merge(customer_df, result_df, on='ID', how='inner')
print(merged_df.head())
#print(len(merged_df['ID'].unique()))

        ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0  5008804           M            Y               Y             0   
1  5008805           M            Y               Y             0   
2  5008806           M            Y               Y             0   
3  5008808           F            N               Y             0   
4  5008810           F            N               Y             0   

   AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0          427500.0               Working               Higher education   
1          427500.0               Working               Higher education   
2          112500.0               Working  Secondary / secondary special   
3          270000.0  Commercial associate  Secondary / secondary special   
4          270000.0  Commercial associate  Secondary / secondary special   

     NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  DAYS_BIRTH  DAYS_EMPLOYED  \
0        Civil marriage   Rented apartment      -12005 

In [20]:
#bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
#labels = ['0-10%', '10-20%', '20-30%', '30-40%', '40-50%', '50-60%', '60-70%', '70-80%', '80-90%', '90-100%']

#bins = [0, 0.2, 0.4, 0.6,  0.8, 1]
#labels = ['1', '2', '3', '4', '5']

# Define bins with 5% steps
#bins = [i/20 for i in range(21)]  # 0, 0.05, 0.1, ..., 0.95, 1

# Define corresponding labels for each bin
#labels = ['0-5%', '5-10%', '10-15%', '15-20%', '20-25%', '25-30%', '30-35%', '35-40%', 
 #         '40-45%', '45-50%', '50-55%', '55-60%', '60-65%', '65-70%', '70-75%', '75-80%',
  #        '80-85%', '85-90%', '90-95%', '95-100%']

#merged_df['overdue_class'] = pd.cut(merged_df['overdue_likelihood'], bins=bins, labels=labels, include_lowest=True)
#merged_df['overdue_class'].value_counts()


#Manual resampling

# Find the number of samples in the smallest class
#min_class_size = merged_df['overdue_class'].value_counts().min()
#print(f"Minimum class size: {min_class_size}")

# Resample each class to have the same number of samples as the smallest class
#resampled_df = merged_df.groupby('overdue_class').apply(lambda x: x.sample(min_class_size, replace=True)).reset_index(drop=True)
#merged_df = resampled_df
# Check the class distribution after resampling
#print(resampled_df['overdue_class'].value_counts())

In [21]:
one_hot_cols = []
ordinal_cols = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_EDUCATION_TYPE']


merged_df = merged_df.drop('OCCUPATION_TYPE', axis=1)
merged_df = merged_df.drop('NAME_HOUSING_TYPE', axis=1)
merged_df = merged_df.drop('NAME_INCOME_TYPE', axis=1)
merged_df = merged_df.drop('NAME_FAMILY_STATUS', axis=1)
#merged_df = merged_df.drop('overdue_likelihood', axis=1)
merged_df = merged_df.drop('ID', axis=1)

df_ord = merged_df.copy()
ordinal_encoder = OrdinalEncoder()
df_ord[ordinal_cols] = ordinal_encoder.fit_transform(df_ord[ordinal_cols])

df_enc = pd.get_dummies(df_ord, columns=one_hot_cols)

print(df_enc.head())

   CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  \
0          1.0           1.0              1.0             0          427500.0   
1          1.0           1.0              1.0             0          427500.0   
2          1.0           1.0              1.0             0          112500.0   
3          0.0           0.0              1.0             0          270000.0   
4          0.0           0.0              1.0             0          270000.0   

   NAME_EDUCATION_TYPE  DAYS_BIRTH  DAYS_EMPLOYED  FLAG_MOBIL  \
0                  1.0      -12005          -4542           1   
1                  1.0      -12005          -4542           1   
2                  4.0      -21474          -1134           1   
3                  4.0      -19110          -3051           1   
4                  4.0      -19110          -3051           1   

   FLAG_WORK_PHONE  FLAG_PHONE  FLAG_EMAIL  CNT_FAM_MEMBERS  \
0                1           0           0              2.0

In [22]:
#X_enc = df_enc.drop('overdue_class', axis=1)
X_enc = df_enc.drop('overdue_likelihood', axis=1)
X_enc = X_enc.drop('FLAG_WORK_PHONE', axis=1)
X_enc = X_enc.drop('FLAG_PHONE', axis=1)
X_enc = X_enc.drop('FLAG_EMAIL', axis=1)
X_enc = X_enc.drop('FLAG_MOBIL', axis=1)
X_enc = X_enc.drop('NAME_EDUCATION_TYPE', axis=1)
Y = df_enc['overdue_likelihood']

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_enc)
df_scaled = pd.DataFrame(X_scaled, columns= X_enc.columns)
print(df_scaled.head())

   CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  \
0          1.0           1.0              1.0           0.0          0.258721   
1          1.0           1.0              1.0           0.0          0.258721   
2          1.0           1.0              1.0           0.0          0.055233   
3          0.0           0.0              1.0           0.0          0.156977   
4          0.0           0.0              1.0           0.0          0.156977   

   DAYS_BIRTH  DAYS_EMPLOYED  CNT_FAM_MEMBERS  
0    0.753539       0.029324         0.052632  
1    0.753539       0.029324         0.052632  
2    0.210810       0.038270         0.052632  
3    0.346306       0.033237         0.000000  
4    0.346306       0.033237         0.000000  


# TODO:
- overdue-likelyhood zu labels
- encoden
    - Eig geht fast alles ordinal encoded
    - OCCUPATION_TYPE fliegt ganz raus
- scalen
- random forest




## Modeling

In [23]:
from sklearn.model_selection import train_test_split, GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(df_scaled, Y, test_size=0.3, random_state=42)

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class SimpleNNRegression(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SimpleNNRegression, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Second hidden layer
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, hidden_size)  # Third hidden layer
        self.relu3 = nn.ReLU()
        #self.fc4 = nn.Linear(input_size, hidden_size)
        #self.relu4 = nn.ReLU()
        #self.fc5 = nn.Linear(hidden_size, hidden_size)  # Second hidden layer
        #self.relu5 = nn.ReLU()
        #self.fc6 = nn.Linear(hidden_size, hidden_size)  # Third hidden layer
        #self.relu6 = nn.ReLU()
        self.fc7 = nn.Linear(hidden_size, 1)  # Output layer for regression
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        #out = self.fc4(x)
        #out = self.relu4(out)
        #out = self.fc5(out)
        #out = self.relu5(out)
        #out = self.fc6(out)
        #out = self.relu6(out)
        out = self.fc7(out)
        return out
    
# Calculate class weights
#class_counts = y_train.value_counts()
#class_weights = [len(y_train) / class_counts[i] for i in range(len(class_counts))]

# Convert class weights to tensor and move to the same device as your model
#class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

# Use the weights in your loss function
#criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
    
# Label encode your target variable if it's categorical
#label_encoder = LabelEncoder()
#y_train_encoded = label_encoder.fit_transform(y_train)
#y_test_encoded = label_encoder.transform(y_test)


# Convert DataFrames to NumPy arrays for features
X_train_np = X_train.values.astype(np.float32)
X_test_np = X_test.values.astype(np.float32)

# For regression, directly use the continuous target variable from y_train and y_test
y_train_np = y_train.values.astype(np.float32)
y_test_np = y_test.values.astype(np.float32)

# Convert feature and target arrays to tensors
X_train_tensor = torch.tensor(X_train_np).to(device)
y_train_tensor = torch.tensor(y_train_np).to(device)
X_test_tensor = torch.tensor(X_test_np).to(device)
y_test_tensor = torch.tensor(y_test_np).to(device)


# Hyperparameters
input_size = X_train.shape[1]
hidden_size = 500  # for example
num_classes = len(np.unique(y_train))
num_epochs = 1000
learning_rate = 0.002

# Create an instance of your model for regression
model = SimpleNNRegression(input_size, hidden_size).to(device)

# Use a regression loss function like MSELoss
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs.squeeze(), y_train_tensor)  # Squeeze to match dimensions

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Evaluation for regression
model.eval()
with torch.no_grad():
    predicted = model(X_test_tensor).squeeze()
    mse = criterion(predicted, y_test_tensor)
    print(f'Mean Squared Error: {mse.item()}')
    
    
    # Evaluation for regression
model.eval()
with torch.no_grad():
    predicted = model(X_test_tensor).squeeze()
    mse = criterion(predicted, y_test_tensor)
    print(f'Mean Squared Error: {mse.item()}')

    # Calculate a custom 'accuracy' metric for regression
    threshold = 0.10  # 10% of the actual value
    within_threshold = torch.abs(predicted - y_test_tensor) <= (threshold * torch.abs(y_test_tensor))
    custom_accuracy = torch.mean(within_threshold.float()) * 100
    print(f'Custom Accuracy (within {threshold*100}% of actual): {custom_accuracy:.2f}%')
    
# Convert the predictions and actual values to a Pandas DataFrame for easier handling
predicted_np = predicted.cpu().numpy()
y_test_np = y_test_tensor.cpu().numpy()
comparison_df = pd.DataFrame({'Actual': y_test_np, 'Predicted': predicted_np})

# Randomly sample some examples from the DataFrame
num_examples_to_show = 10  # You can change this number
random_examples = comparison_df.sample(n=num_examples_to_show)

print(random_examples)

Using device: cuda
Epoch [1/3000], Loss: 0.5825
Epoch [2/3000], Loss: 2.4994
Epoch [3/3000], Loss: 0.3509
Epoch [4/3000], Loss: 0.4840
Epoch [5/3000], Loss: 0.5182
Epoch [6/3000], Loss: 0.5143
Epoch [7/3000], Loss: 0.4925
Epoch [8/3000], Loss: 0.4572
Epoch [9/3000], Loss: 0.4134
Epoch [10/3000], Loss: 0.3715
Epoch [11/3000], Loss: 0.3463
Epoch [12/3000], Loss: 0.3603
Epoch [13/3000], Loss: 0.3862
Epoch [14/3000], Loss: 0.3477
Epoch [15/3000], Loss: 0.3426
Epoch [16/3000], Loss: 0.3436
Epoch [17/3000], Loss: 0.3473
Epoch [18/3000], Loss: 0.3514
Epoch [19/3000], Loss: 0.3540
Epoch [20/3000], Loss: 0.3545
Epoch [21/3000], Loss: 0.3529
Epoch [22/3000], Loss: 0.3498
Epoch [23/3000], Loss: 0.3462
Epoch [24/3000], Loss: 0.3433
Epoch [25/3000], Loss: 0.3418
Epoch [26/3000], Loss: 0.3416
Epoch [27/3000], Loss: 0.3428
Epoch [28/3000], Loss: 0.3444
Epoch [29/3000], Loss: 0.3456
Epoch [30/3000], Loss: 0.3458
Epoch [31/3000], Loss: 0.3450
Epoch [32/3000], Loss: 0.3438
Epoch [33/3000], Loss: 0.3425


Epoch [270/3000], Loss: 0.3399
Epoch [271/3000], Loss: 0.3398
Epoch [272/3000], Loss: 0.3401
Epoch [273/3000], Loss: 0.3404
Epoch [274/3000], Loss: 0.3404
Epoch [275/3000], Loss: 0.3401
Epoch [276/3000], Loss: 0.3396
Epoch [277/3000], Loss: 0.3395
Epoch [278/3000], Loss: 0.3396
Epoch [279/3000], Loss: 0.3398
Epoch [280/3000], Loss: 0.3398
Epoch [281/3000], Loss: 0.3396
Epoch [282/3000], Loss: 0.3393
Epoch [283/3000], Loss: 0.3392
Epoch [284/3000], Loss: 0.3392
Epoch [285/3000], Loss: 0.3393
Epoch [286/3000], Loss: 0.3393
Epoch [287/3000], Loss: 0.3391
Epoch [288/3000], Loss: 0.3390
Epoch [289/3000], Loss: 0.3389
Epoch [290/3000], Loss: 0.3389
Epoch [291/3000], Loss: 0.3389
Epoch [292/3000], Loss: 0.3389
Epoch [293/3000], Loss: 0.3388
Epoch [294/3000], Loss: 0.3386
Epoch [295/3000], Loss: 0.3386
Epoch [296/3000], Loss: 0.3386
Epoch [297/3000], Loss: 0.3385
Epoch [298/3000], Loss: 0.3384
Epoch [299/3000], Loss: 0.3383
Epoch [300/3000], Loss: 0.3383
Epoch [301/3000], Loss: 0.3382
Epoch [3

Epoch [540/3000], Loss: 0.3270
Epoch [541/3000], Loss: 0.3269
Epoch [542/3000], Loss: 0.3270
Epoch [543/3000], Loss: 0.3272
Epoch [544/3000], Loss: 0.3280
Epoch [545/3000], Loss: 0.3292
Epoch [546/3000], Loss: 0.3302
Epoch [547/3000], Loss: 0.3271
Epoch [548/3000], Loss: 0.3304
Epoch [549/3000], Loss: 0.3331
Epoch [550/3000], Loss: 0.3276
Epoch [551/3000], Loss: 0.3359
Epoch [552/3000], Loss: 0.3355
Epoch [553/3000], Loss: 0.3397
Epoch [554/3000], Loss: 0.3366
Epoch [555/3000], Loss: 0.3339
Epoch [556/3000], Loss: 0.3374
Epoch [557/3000], Loss: 0.3323
Epoch [558/3000], Loss: 0.3323
Epoch [559/3000], Loss: 0.3339
Epoch [560/3000], Loss: 0.3322
Epoch [561/3000], Loss: 0.3324
Epoch [562/3000], Loss: 0.3298
Epoch [563/3000], Loss: 0.3318
Epoch [564/3000], Loss: 0.3308
Epoch [565/3000], Loss: 0.3303
Epoch [566/3000], Loss: 0.3288
Epoch [567/3000], Loss: 0.3288
Epoch [568/3000], Loss: 0.3290
Epoch [569/3000], Loss: 0.3292
Epoch [570/3000], Loss: 0.3294
Epoch [571/3000], Loss: 0.3282
Epoch [5

Epoch [810/3000], Loss: 0.3304
Epoch [811/3000], Loss: 0.3303
Epoch [812/3000], Loss: 0.3304
Epoch [813/3000], Loss: 0.3310
Epoch [814/3000], Loss: 0.3330
Epoch [815/3000], Loss: 0.3300
Epoch [816/3000], Loss: 0.3348
Epoch [817/3000], Loss: 0.3373
Epoch [818/3000], Loss: 0.3383
Epoch [819/3000], Loss: 0.3303
Epoch [820/3000], Loss: 0.3432
Epoch [821/3000], Loss: 0.3358
Epoch [822/3000], Loss: 0.3475
Epoch [823/3000], Loss: 0.3384
Epoch [824/3000], Loss: 0.3324
Epoch [825/3000], Loss: 0.3486
Epoch [826/3000], Loss: 0.3329
Epoch [827/3000], Loss: 0.3437
Epoch [828/3000], Loss: 0.3457
Epoch [829/3000], Loss: 0.3373
Epoch [830/3000], Loss: 0.3330
Epoch [831/3000], Loss: 0.3432
Epoch [832/3000], Loss: 0.3333
Epoch [833/3000], Loss: 0.3338
Epoch [834/3000], Loss: 0.3368
Epoch [835/3000], Loss: 0.3375
Epoch [836/3000], Loss: 0.3353
Epoch [837/3000], Loss: 0.3336
Epoch [838/3000], Loss: 0.3347
Epoch [839/3000], Loss: 0.3349
Epoch [840/3000], Loss: 0.3327
Epoch [841/3000], Loss: 0.3327
Epoch [8

Epoch [1080/3000], Loss: 0.3206
Epoch [1081/3000], Loss: 0.3237
Epoch [1082/3000], Loss: 0.3199
Epoch [1083/3000], Loss: 0.3226
Epoch [1084/3000], Loss: 0.3211
Epoch [1085/3000], Loss: 0.3191
Epoch [1086/3000], Loss: 0.3209
Epoch [1087/3000], Loss: 0.3206
Epoch [1088/3000], Loss: 0.3194
Epoch [1089/3000], Loss: 0.3189
Epoch [1090/3000], Loss: 0.3193
Epoch [1091/3000], Loss: 0.3195
Epoch [1092/3000], Loss: 0.3187
Epoch [1093/3000], Loss: 0.3191
Epoch [1094/3000], Loss: 0.3197
Epoch [1095/3000], Loss: 0.3197
Epoch [1096/3000], Loss: 0.3196
Epoch [1097/3000], Loss: 0.3190
Epoch [1098/3000], Loss: 0.3184
Epoch [1099/3000], Loss: 0.3184
Epoch [1100/3000], Loss: 0.3188
Epoch [1101/3000], Loss: 0.3198
Epoch [1102/3000], Loss: 0.3200
Epoch [1103/3000], Loss: 0.3206
Epoch [1104/3000], Loss: 0.3191
Epoch [1105/3000], Loss: 0.3185
Epoch [1106/3000], Loss: 0.3197
Epoch [1107/3000], Loss: 0.3196
Epoch [1108/3000], Loss: 0.3186
Epoch [1109/3000], Loss: 0.3179
Epoch [1110/3000], Loss: 0.3182
Epoch [1

...

Epoch [1863/3000], Loss: 0.3076
Epoch [1864/3000], Loss: 0.3131
Epoch [1865/3000], Loss: 0.3075
Epoch [1866/3000], Loss: 0.3066
Epoch [1867/3000], Loss: 0.3090
Epoch [1868/3000], Loss: 0.3064
Epoch [1869/3000], Loss: 0.3073
Epoch [1870/3000], Loss: 0.3087
Epoch [1871/3000], Loss: 0.3059
Epoch [1872/3000], Loss: 0.3064
Epoch [1873/3000], Loss: 0.3098
Epoch [1874/3000], Loss: 0.3084
Epoch [1875/3000], Loss: 0.3062
Epoch [1876/3000], Loss: 0.3075
Epoch [1877/3000], Loss: 0.3065
Epoch [1878/3000], Loss: 0.3044
Epoch [1879/3000], Loss: 0.3064
Epoch [1880/3000], Loss: 0.3095
Epoch [1881/3000], Loss: 0.3096
Epoch [1882/3000], Loss: 0.3051
Epoch [1883/3000], Loss: 0.3106
Epoch [1884/3000], Loss: 0.3112
Epoch [1885/3000], Loss: 0.3070
Epoch [1886/3000], Loss: 0.3150
Epoch [1887/3000], Loss: 0.3116
Epoch [1888/3000], Loss: 0.3051
Epoch [1889/3000], Loss: 0.3095
Epoch [1890/3000], Loss: 0.3087
Epoch [1891/3000], Loss: 0.3071
Epoch [1892/3000], Loss: 0.3093
Epoch [1893/3000], Loss: 0.3058
Epoch [1

Epoch [2124/3000], Loss: 0.3036
Epoch [2125/3000], Loss: 0.3032
Epoch [2126/3000], Loss: 0.3031
Epoch [2127/3000], Loss: 0.3037
Epoch [2128/3000], Loss: 0.3026
Epoch [2129/3000], Loss: 0.3009
Epoch [2130/3000], Loss: 0.3021
Epoch [2131/3000], Loss: 0.3039
Epoch [2132/3000], Loss: 0.3056
Epoch [2133/3000], Loss: 0.3070
Epoch [2134/3000], Loss: 0.3045
Epoch [2135/3000], Loss: 0.3105
Epoch [2136/3000], Loss: 0.3080
Epoch [2137/3000], Loss: 0.3048
Epoch [2138/3000], Loss: 0.3109
Epoch [2139/3000], Loss: 0.3116
Epoch [2140/3000], Loss: 0.3028
Epoch [2141/3000], Loss: 0.3038
Epoch [2142/3000], Loss: 0.3052
Epoch [2143/3000], Loss: 0.3026
Epoch [2144/3000], Loss: 0.3053
Epoch [2145/3000], Loss: 0.3042
Epoch [2146/3000], Loss: 0.3013
Epoch [2147/3000], Loss: 0.3065
Epoch [2148/3000], Loss: 0.3091
Epoch [2149/3000], Loss: 0.3021
Epoch [2150/3000], Loss: 0.3069
Epoch [2151/3000], Loss: 0.3084
Epoch [2152/3000], Loss: 0.3049
Epoch [2153/3000], Loss: 0.3110
Epoch [2154/3000], Loss: 0.3047
Epoch [2

Epoch [2385/3000], Loss: 0.3013
Epoch [2386/3000], Loss: 0.3047
Epoch [2387/3000], Loss: 0.2977
Epoch [2388/3000], Loss: 0.3031
Epoch [2389/3000], Loss: 0.3148
Epoch [2390/3000], Loss: 0.3010
Epoch [2391/3000], Loss: 0.3106
Epoch [2392/3000], Loss: 0.3158
Epoch [2393/3000], Loss: 0.3165
Epoch [2394/3000], Loss: 0.3093
Epoch [2395/3000], Loss: 0.3113
Epoch [2396/3000], Loss: 0.3095
Epoch [2397/3000], Loss: 0.3102
Epoch [2398/3000], Loss: 0.3061
Epoch [2399/3000], Loss: 0.3057
Epoch [2400/3000], Loss: 0.3081
Epoch [2401/3000], Loss: 0.3031
Epoch [2402/3000], Loss: 0.3030
Epoch [2403/3000], Loss: 0.3034
Epoch [2404/3000], Loss: 0.3033
Epoch [2405/3000], Loss: 0.3034
Epoch [2406/3000], Loss: 0.3005
Epoch [2407/3000], Loss: 0.2993
Epoch [2408/3000], Loss: 0.3015
Epoch [2409/3000], Loss: 0.3007
Epoch [2410/3000], Loss: 0.2985
Epoch [2411/3000], Loss: 0.2991
Epoch [2412/3000], Loss: 0.2998
Epoch [2413/3000], Loss: 0.2993
Epoch [2414/3000], Loss: 0.2973
Epoch [2415/3000], Loss: 0.2983
Epoch [2

Epoch [2646/3000], Loss: 0.2970
Epoch [2647/3000], Loss: 0.2991
Epoch [2648/3000], Loss: 0.2973
Epoch [2649/3000], Loss: 0.2982
Epoch [2650/3000], Loss: 0.2963
Epoch [2651/3000], Loss: 0.2981
Epoch [2652/3000], Loss: 0.2960
Epoch [2653/3000], Loss: 0.2968
Epoch [2654/3000], Loss: 0.2975
Epoch [2655/3000], Loss: 0.3000
Epoch [2656/3000], Loss: 0.2992
Epoch [2657/3000], Loss: 0.2955
Epoch [2658/3000], Loss: 0.3001
Epoch [2659/3000], Loss: 0.3032
Epoch [2660/3000], Loss: 0.2959
Epoch [2661/3000], Loss: 0.2997
Epoch [2662/3000], Loss: 0.3036
Epoch [2663/3000], Loss: 0.2980
Epoch [2664/3000], Loss: 0.3080
Epoch [2665/3000], Loss: 0.3048
Epoch [2666/3000], Loss: 0.2983
Epoch [2667/3000], Loss: 0.3048
Epoch [2668/3000], Loss: 0.2990
Epoch [2669/3000], Loss: 0.3009
Epoch [2670/3000], Loss: 0.2987
Epoch [2671/3000], Loss: 0.2970
Epoch [2672/3000], Loss: 0.2987
Epoch [2673/3000], Loss: 0.2964
Epoch [2674/3000], Loss: 0.2965
Epoch [2675/3000], Loss: 0.2974
Epoch [2676/3000], Loss: 0.2957
Epoch [2

Epoch [2907/3000], Loss: 0.2933
Epoch [2908/3000], Loss: 0.2923
Epoch [2909/3000], Loss: 0.2916
Epoch [2910/3000], Loss: 0.2915
Epoch [2911/3000], Loss: 0.2927
Epoch [2912/3000], Loss: 0.2919
Epoch [2913/3000], Loss: 0.2921
Epoch [2914/3000], Loss: 0.2924
Epoch [2915/3000], Loss: 0.2942
Epoch [2916/3000], Loss: 0.2919
Epoch [2917/3000], Loss: 0.2918
Epoch [2918/3000], Loss: 0.2905
Epoch [2919/3000], Loss: 0.2920
Epoch [2920/3000], Loss: 0.2916
Epoch [2921/3000], Loss: 0.2927
Epoch [2922/3000], Loss: 0.2941
Epoch [2923/3000], Loss: 0.2954
Epoch [2924/3000], Loss: 0.2942
Epoch [2925/3000], Loss: 0.2947
Epoch [2926/3000], Loss: 0.2922
Epoch [2927/3000], Loss: 0.2932
Epoch [2928/3000], Loss: 0.2918
Epoch [2929/3000], Loss: 0.2922
Epoch [2930/3000], Loss: 0.2912
Epoch [2931/3000], Loss: 0.2913
Epoch [2932/3000], Loss: 0.2927
Epoch [2933/3000], Loss: 0.2947
Epoch [2934/3000], Loss: 0.2981
Epoch [2935/3000], Loss: 0.2945
Epoch [2936/3000], Loss: 0.2920
Epoch [2937/3000], Loss: 0.2984
Epoch [2

## Evaluation